In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

데이터 불러오기, 분할

In [2]:
train_df = pd.read_csv('final_train_data.csv')
test_df = pd.read_csv('final_test_data.csv')

In [3]:
#train_data, valid_data, test_data로 분할
train_data = train_df[train_df.date <= '2016-07-31']
valid_data = train_df[train_df.date >= '2016-08-01']
test_data=test_df

In [4]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.model_selection import GridSearchCV



In [5]:
train_data.head()

,date,store_nbr,family,sales,onpromotion,dcoilwtico,transactions,type_y,cluster,Year,Month,Weekday,trend,NewYearsDay,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre,no_work_day
0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
1,2013-01-01,1,BABY CARE,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
2,2013-01-01,1,BEAUTY,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
3,2013-01-01,1,BEVERAGES,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True
4,2013-01-01,1,BOOKS,0.0,0,93.14,1420.876311,D,13,2013,1,1,0,True,0,0,0,0,0,True


데이터 전처리

In [6]:
#date 는 Year,Month 존재하므로 drop
train_data.drop(columns = ['date','NewYearsDay','no_work_day','cluster'], inplace = True)
valid_data.drop(columns = ['date','NewYearsDay','no_work_day','cluster'], inplace = True)
test_data.drop(columns = ['date','NewYearsDay','no_work_day','cluster'], inplace = True)

In [7]:
y_train=train_data['sales']
X_train=train_data.drop(columns='sales')

X_valid_data = valid_data.drop(columns = ['sales'])
y_valid_data = valid_data['sales']


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder


#매개 변수 설정과 데이터 스케일에 매우 민감 : 입력 특성의 범위가 비슷해야하기 때문에 모든 특성 값을 0과 1 사이로 맞추는 방법
preproc = ColumnTransformer(
    [('standard', StandardScaler(), ['transactions']),
     ('minmax', MinMaxScaler(), ['onpromotion', 'dcoilwtico']),
     ('cat', OneHotEncoder(sparse = False, drop = 'first'), ['store_nbr', 'family', 'type_y'])],
     remainder = 'passthrough')



In [9]:
preproc.fit(X_train)
cat_features_names = preproc.named_transformers_['cat'].get_feature_names()
labels = np.concatenate([['transactions','onpromotion', 'dcoilwtico'], cat_features_names, ['Year', 'Month', 'Weekday', 'trend','Earthquake', 'Cyber Monday', 'Football', 'Black Friday','Dia de la Madre']])

X_train_data_trans = pd.DataFrame(preproc.transform(X_train), columns = labels)
X_valid_data_trans = pd.DataFrame(preproc.transform(X_valid_data), columns = labels)


In [10]:
X_train_data_trans.head()

,transactions,onpromotion,dcoilwtico,x0_2,x0_3,x0_4,x0_5,x0_6,x0_7,x0_8,...,x2_E,Year,Month,Weekday,trend,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre
0,-0.269291,0.0,0.792965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.269291,0.0,0.792965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.269291,0.0,0.792965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.269291,0.0,0.792965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.269291,0.0,0.792965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test_data

,store_nbr,family,onpromotion,dcoilwtico,transactions,type_y,Year,Month,Weekday,trend,Earthquake,Cyber Monday,Football,Black Friday,Dia de la Madre
0,1,AUTOMOTIVE,0,46.80,1448.259579,D,2017,8,2,1688,0,0,0,0,0
1,1,BABY CARE,0,46.80,1448.259579,D,2017,8,2,1688,0,0,0,0,0
2,1,BEAUTY,2,46.80,1448.259579,D,2017,8,2,1688,0,0,0,0,0
3,1,BEVERAGES,20,46.80,1448.259579,D,2017,8,2,1688,0,0,0,0,0
4,1,BOOKS,0,46.80,1448.259579,D,2017,8,2,1688,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,9,POULTRY,1,47.26,1986.976875,B,2017,8,3,1703,0,0,0,0,0
28508,9,PREPARED FOODS,0,47.26,1986.976875,B,2017,8,3,1703,0,0,0,0,0
28509,9,PRODUCE,1,47.26,1986.976875,B,2017,8,3,1703,0,0,0,0,0
28510,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,1986.976875,B,2017,8,3,1703,0,0,0,0,0


modelling

In [ ]:
import matplotlib.pyplot as plot
import numpy as np
import math

from sklearn.svm import SVR

model = SVR(kernel = 'rbf')
model.fit(X_train_data_trans, y_train)

  # Create Support Vector Regression model
  # kernel : 선형 커널
  # C : 학습 오류에 대한 패널티, C 값이 클 수록 모델이 학습 데이터에 좀 더 최적화 됨, 너무 크면 오버피팅 발생
  # Epsilon : 임계값, 예측한 값이 GT 범위 안에 있으면 패널티 부여 X


In [ ]:
relation_square = model.score(X_valid_data_trans, y_valid_data)
print('결정계수 R : ', relation_square)


In [ ]:
y_pred = sv_regressor.predict(X_valid_data_trans)


In [ ]:
from sklearn.metrics import mean_squared_error, explained_variance_score
mse = mean_squared_error(y_valid_data, y_pred)
evs = explained_variance_score(y_valid_data, y_pred)


rmsle

In [ ]:
train_pred = model.predict(X_train_data_trans)
valid_pred = model.predict(X_valid_data_trans)

train_pred[train_pred < 0] = 0
valid_pred[valid_pred < 0] = 0


In [ ]:
from sklearn.metrics import mean_squared_log_error
print('훈련데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_train, train_pred)))
print('검증데이터 rmsle: ', np.sqrt(mean_squared_log_error(y_valid_data, valid_pred)))

submit

In [ ]:
test_pred = model.predict(test_data)
test_pred[test_pred < 0] = 0
                          
submission = pd.read_csv('C:/Users/Kim Eun ho/Desktop/sooktat/프로젝트/sample_submission.csv')
submission['sales'] = test_pred
submission.to_csv('submission.csv', index = False)